# Roberta

In [1]:
 !pip install -q torch

In [2]:
!pip install -q transformers

In [10]:
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch

In [4]:
QA_input = [ { 'question' : 'What is the statistical distribution of fission products resulting from the fission?',
              'context': 'Neutrinos & Non-proliferation in Europe Michel Cribier* APC, Paris CEA/Saclay, DAPNIA/SPP The International Atomic Energy Agency (IAEA) is the United Nations agency in charge of the development of peaceful use of atomic energy. In particular IAEA is the verification authority of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT). To do that jobs inspections of civil nuclear installations and related facilities under safeguards agreements are made in more than 140 states. IAEA uses many different tools for these verifications, like neutron monitor, gamma spectroscopy, but also bookeeping of the isotopic composition at the fuel element level before and after their use in the nuclear power station. In particular it verifie that weapon-origin and other fissile materials that Russia and USA have released from their defense programmes are used for civil application. The existence of an antineutrino signal sensitive to the power and to the isotopic composition of a reactor core, as first proposed by Mikaelian et al. and as demonstrated by the Bugey and Rovno experiments, , could provide a means to address certain safeguards applications. Thus the IAEA recently ask members states to make a feasibility study to determine whether antineutrino detection methods might provide practical safeguards tools for selected applications. If this method proves to be useful, IAEA has the power to decide that any new nuclear power plants built has to include an antineutrino monitor. Within the Double Chooz collaboration, an experiment mainly devoted to study the fundamental properties of neutrinos, we thought that we were in a good position to evaluate the interest of using antineutrino detection to remotely monitor nuclear power station. This effort in Europe, supplemented by the US effort , will constitute the basic answer to IAEA of the neutrino community. * On behalf of a collective work by S. Cormon, M. Fallot, H. Faust, T. Lasserre, A. Letourneau, D. Lhuillier, V. Sinev from DAPNIA, Subatech and ILL. Figure 1 : The statistical distribution of the fission products resulting from the fission of the most important fissile nuclei 235U and 239Pu shows two humps, one centered around masses 100 and the other one centered around 135.'},
             {'question': 'The low mass hump is at higher mass in what fission?',
              'context':'The low mass hump is at higher mass in 239Pu fission than in 235U, resulting in different nuclei and decays. The high penetration power of antineutrinos and the detection capability might provide a means to make remote, non-intrusive measurements of plutonium content in reactors . The antineutrino flux and energy spectrum depends upon the thermal power and on the fissile isotopic composition of the reactor fuel. Indeed, when a heavy nuclei (Uranium, Plutonium) experience a fission, it produce two unequal fission fragments (and a few free neutrons) ; the statistical distribution of the atomic masses is depicted in figure 1. All these nuclei immediately produced are extremely unstable - they are too rich in neutrons - and thus ÃŸ decay toward stable nuclei with an average of 6 ÃŸ decays. All these process involving several hundreds of unstable nuclei, with their excited states, makes very difficult to understand details of the physics, moreover, the most energetic antineutrinos, which are detected more easily, are produced in the very first decays, involving nuclei with typical lifetime smaller than a second. 235U 239Pu released energy per fission 201.7 MeV 210.0 MeV Mean energy of Î½ 2.94 MeV 2.84 MeV Î½ per fission > 1.8 MeV 1.92 1.45 average inter.'}]

In [5]:
model_name = 'deepset/roberta-base-squad2'

In [6]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\engba\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
input0 = tokenizer(QA_input[0]['question'], QA_input[0]['context'], return_tensors='pt')
output0 =model(**input0)

In [12]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

for qa in QA_input:
    question = qa['question']
    context = qa['context']
    answer = answer_question(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What is the statistical distribution of fission products resulting from the fission?
Answer: <s>

Question: The low mass hump is at higher mass in what fission?
Answer: 



In [13]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer with the argmax of the score
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

    return answer

for qa in QA_input:
    question = qa['question']
    context = qa['context']
    answer = answer_question(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What is the statistical distribution of fission products resulting from the fission?
Answer: <s>

Question: The low mass hump is at higher mass in what fission?
Answer: 



## Second Method

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering
# Downloading Model
qa_model_name = 'deepset/roberta-base-squad2'
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)

t5_model_name = 't5-base'
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
# Question Generator
def generate_questions(context):
    input_text = f"generate questions: {context}"
    inputs = t5_tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)
    outputs = t5_model.generate(inputs['input_ids'], max_length=512, num_return_sequences=5, num_beams=5)
    questions = [t5_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return questions

# Answer Generator
def answer_question(question, context):
    inputs = qa_tokenizer(question, context, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        outputs = qa_model(input_ids=input_ids, attention_mask=attention_mask)
    
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

    return answer

def generate_qas(context):
    questions = generate_questions(context)
    qa_pairs = []
    for question in questions:
        answer = answer_question(question, context)
        qa_pairs.append({'question': question, 'answer': answer})
    return qa_pairs

# Example Context
context = """
Neutrinos & Non-proliferation in Europe Michel Cribier* APC, Paris CEA/Saclay, DAPNIA/SPP The International Atomic Energy Agency (IAEA) is the United Nations agency in charge of the development of peaceful use of atomic energy. In particular IAEA is the verification authority of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT). To do that jobs inspections of civil nuclear installations and related facilities under safeguards agreements are made in more than 140 states. IAEA uses many different tools for these verifications, like neutron monitor, gamma spectroscopy, but also bookeeping of the isotopic composition at the fuel element level before and after their use in the nuclear power station. In particular it verifie that weapon-origin and other fissile materials that Russia and USA have released from their defense programmes are used for civil application. The existence of an antineutrino signal sensitive to the power and to the isotopic composition of a reactor core, as first proposed by Mikaelian et al. and as demonstrated by the Bugey and Rovno experiments, , could provide a means to address certain safeguards applications. Thus the IAEA recently ask members states to make a feasibility study to determine whether antineutrino detection methods might provide practical safeguards tools for selected applications. If this method proves to be useful, IAEA has the power to decide that any new nuclear power plants built has to include an antineutrino monitor. Within the Double Chooz collaboration, an experiment mainly devoted to study the fundamental properties of neutrinos, we thought that we were in a good position to evaluate the interest of using antineutrino detection to remotely monitor nuclear power station. This effort in Europe, supplemented by the US effort , will constitute the basic answer to IAEA of the neutrino community. * On behalf of a collective work by S. Cormon, M. Fallot, H. Faust, T. Lasserre, A. Letourneau, D. Lhuillier, V. Sinev from DAPNIA, Subatech and ILL. Figure 1 : The statistical distribution of the fission products resulting from the fission of the most important fissile nuclei 235U and 239Pu shows two humps, one centered around masses 100 and the other one centered around 135.
"""

# QA 
qa_pairs = generate_qas(context)
for qa in qa_pairs:
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answer']}\n")

Question: antineutrino signal sensitive to the power and isotopic composition of a reactor core. double Chooz: we thought that we were in a good position to evaluate the interest of using antineutrino detection to remotely monitor nuclear power station. figure 1 : The statistical distribution of the fission products resulting from the fission.
Answer: <s>

Question: antineutrino signal sensitive to the power and isotopic composition of a reactor core. double Chooz: we thought that we were in a good position to evaluate the interest of using antineutrino detection to remotely monitor nuclear power station. figure 1 : The statistical distribution of the fission products resulting from the fission process.
Answer: <s>

Question: antineutrino signal sensitive to the power and isotopic composition of a reactor core. double Chooz: we thought that we were in a good position to evaluate the interest of using antineutrino detection to remotely monitor nuclear power station. figure 1 : The stati